# Полиномиальная регрессия. Кросс-валидация.

In [ ]:
import numpy as np
import random
from matplotlib import pyplot as plt

from sklearn.metrics import mean_squared_error
from sklearn.metrics import mean_absolute_error

Функции для распознавания

In [ ]:
def func(x):
    return np.cos(2 * np.pi * x)
    # return 5 * (x ** 3) + x ** 2 + 5
    # return np.sin(2 * np.pi * x) ** 2
    # return (x ** 5) - 2 * x ** 3 + 1

Функция для построения полинома

In [ ]:
def polynom(x, w):
    y = 0
    for i in range(len(w)):
        y += w[i] * (x ** i)
    return y

Равномерное распределение ошибки

In [ ]:
def uniform_epsilon():
    return random.uniform(-0.3, 0.3)

Генерация нормально распределенной величины

<img src="img_2.png">

In [ ]:
def normal_epsilon(mu=0.0, sigma=0.3, n = 12):
    Hi_array = np.random.uniform(0,1,n)
    # print(Hi_array)
    Sn = np.sum(Hi_array)
    # print(Sn)
    nu = (Sn - (n/2))/(np.sqrt(n/12))
    # print(nu)
    epsilon = mu + nu*sigma
    # print(epsilon)
    
    return epsilon
    

Генерация случайно ошибки

In [ ]:
def error_uniform(x):
    return func(x) + uniform_epsilon()

In [ ]:
def error_normal(x):
    return func(x) + normal_epsilon()

Пример равномерно распределенной случайной величины

In [ ]:
print(uniform_epsilon())

Пример нормально распределенной случайно величины

In [ ]:
print(normal_epsilon())

Генерация точек функциональной зависимости для обучения

In [ ]:
def gen(n, error_func, proportion=0.8):
    x = np.sort(np.random.uniform(0, 1., n))
    set_size = len(x)
    training_size = int(set_size * proportion)

    x_training = x[:training_size+1]
    y_training = np.array(list(map(error_func, x_training)))

    x_test = x[training_size+1:]
    y_test = np.array(list(map(error_func, x_test)))

    return x_training, y_training, x_test, y_test

Генерация точек для кросс-валидации

In [ ]:
def gen_cross_validation(error_func, k):
    x_training = []
    y_training = []
    
    for i_k in range(0, k):
        x_training_k = np.sort(np.random.uniform(0 + i_k*1/k, 0 + (i_k+1)*1/k, k))
        y_training_k = np.array(list(map(error_func, x_training_k)))
        x_training.append(x_training_k)
        y_training.append(y_training_k)

    return x_training, y_training

Поиск весов для регуляризации

<img src="img.png">

In [ ]:
def calculate_weights(x, y, M, l):
    A = []
    b = []

    for i in range(M + 1):
        b.append(np.sum((x ** i) * y))
        a = []

        for j in range(M + 1):
            a.append(np.sum(x ** (i + j)))
            if i == j:
                a[j] += l
        A.append(a)

    return np.linalg.solve(A, b)

Ошибка для кросс-валидации

<img src="img_1.png">

In [ ]:
def err_l(y, f, w, l):
    return (1/2) * np.sum((y - f) ** 2) + (l/2) * np.sum(np.abs(w))

Генерация выборки

In [ ]:
# N = 20
# X_train, y_train, X_test, y_test = gen(N, error_uniform)
X_train, y_train = gen_cross_validation(error_uniform, 5)

In [ ]:
# z = np.linspace(0., 1., 100)
# plt.title('Изначальная функция со случайными ошибками')
# plt.plot(z, func(z), 'b-')
# plt.plot(X_train, y_train, 'ro')
# plt.plot(X_test, y_test, 'ro')
# plt.xlim(0, 1)
# plt.ylim(-2, 2)
# plt.show()

In [ ]:
# results = []
# for l in np.arange(0, 0.03, 0.001):
#     for j in range(0, 16):
#         w = calculate_weights(X_train, y_train, j, l)
#         predict = np.array(list(map(lambda x: polynom(x=x, w=w), X_test)))
#         mse = mean_squared_error(y_test, predict)
#         mae = mean_absolute_error(y_test, predict)
#         results.append((j, l, mse, mae, err_l(y_test, predict, w, l)))
# 
#         plt.title('M = ' + str(j) + ';\n l = ' + str(l) + ';\n MSE = ' + str(mse) + ';\n MAE = ' + str(mae) + ';\n E_l = ' + str(err_l(y_test, predict, w, l)))
# 
#         plt.plot(z, func(z), 'b-')
#         plt.plot(z, polynom(z, w), 'g-')
#         plt.plot(X_train, y_train, 'ro')
#         plt.plot(X_test, y_test, 'ro')
#         plt.xlim(0, 1)
#         plt.ylim(-2, 2)
#         plt.show()

In [ ]:
results = []
k = 5

z = np.linspace(0., 1., 100)
plt.title('Изначальная функция со случайными ошибками')
plt.plot(z, func(z), 'b-')
plt.plot(np.concatenate(X_train), np.concatenate(y_train), 'ro')
plt.xlim(0, 1)
plt.ylim(-2, 2)
plt.show()

for l in np.arange(0, 0.03, 0.004):
    for j in range(0, 16):
        mse_avg = []
        mae_avg = []
        err_l_avg = []
        for i in range(0, k):

            #Выбираем тестовые выборки
            X_test_k = X_train[(k-1)-i]
            y_test_k = y_train[(k-1)-i]

            #Исключаем из тренировочных выборок
            X_train_k = np.concatenate([np.delete(X_train, (k-1)-i)])
            y_train_k = np.concatenate([np.delete(y_train, (k-1)-i)])

            w = calculate_weights(X_train_k, y_train_k, j, l)
            predict = np.array(list(map(lambda x: polynom(x=x, w=w), X_test_k)))

            mse = mean_squared_error(y_test_k, predict)
            mse_avg.append(mse)

            mae = mean_absolute_error(y_test_k, predict)
            mae_avg.append(mae)

            err_lambda = err_l(y_test_k, predict, w, l)
            err_l_avg.append(err_lambda)

            plt.title('M = ' + str(j) + ';\n l = ' + str(l) + ';\n MSE = ' + str(mse) + ';\n MAE = ' + str(mae) + ';\n E_l = ' + str(err_lambda) + ';\n k = ' + str(i))

            plt.plot(z, func(z), 'b-')
            plt.plot(z, polynom(z, w), 'g-')
            plt.plot(X_train_k, y_train_k, 'ro')
            plt.plot(X_test_k, y_test_k, 'yo')
            plt.xlim(0, 1)
            plt.ylim(-2, 2)
            plt.show()

        results.append((j, l, np.average(mse_avg), np.average(mae_avg), np.average(err_l_avg)))

Лучшие параметры регрессии M и l

Для серднеквадратичной ошибки

In [ ]:
for t in (sorted(results, key=lambda t: t[2])):
    print('Degree = ', t[0], ' L = ', t[1], ' MSE = ', t[2])

Для абсолютной ошибки

In [ ]:
for t in (sorted(results, key=lambda t: t[3])):
    print('Degree = ', t[0], ' L = ', t[1], 'MAE = ', t[3])

Для ошибки кросс-валидации

In [ ]:
for t in (sorted(results, key=lambda t: t[4])):
    print('Degree = ', t[0], ' L = ', t[1], 'E_l = ', t[4])

Генерация нормально распределенной ошибки

In [ ]:
# N = 20
# X_train, y_train, X_test, y_test = gen(N, error_normal)
X_train, y_train = gen_cross_validation(error_normal, 5)

In [ ]:
# z = np.linspace(0., 1., 100)
# plt.title('Изначальная функция со случайными ошибками')
# plt.plot(z, func(z), 'b-')
# plt.plot(X_train, y_train, 'ro')
# plt.plot(X_test, y_test, 'ro')
# plt.xlim(0, 1)
# plt.ylim(-2, 2)
# plt.show()

In [ ]:
# results = []
# for l in np.arange(0, 0.03, 0.001):
#     for j in range(1, 15, 1):
#         w = calculate_weights(X_train, y_train, j, l)
#         predict = np.array(list(map(lambda x: polynom(x=x, w=w), X_test)))
#         mse = mean_squared_error(y_test, predict)
#         mae = mean_absolute_error(y_test, predict)
#         results.append((j, l, mse, mae, err_l(y_test, predict, w, l)))
# 
#         plt.title('M = ' + str(j) + ';\n l = ' + str(l) + ';\n MSE = ' + str(mse) + ';\n MAE = ' + str(mae) + ';\n E_l = ' + str(err_l(y_test, predict, w, l)))
#         plt.plot(z, func(z), 'b-')
#         plt.plot(z, polynom(z, w), 'g-')
#         plt.plot(X_train, y_train, 'ro')
#         plt.plot(X_test, y_test, 'ro')
#         plt.xlim(0, 1)
#         plt.ylim(-2, 2)
#         plt.show()

In [ ]:
results = []
k = 5

z = np.linspace(0., 1., 100)
plt.title('Изначальная функция со случайными ошибками')
plt.plot(z, func(z), 'b-')
plt.plot(np.concatenate(X_train), np.concatenate(y_train), 'ro')
plt.xlim(0, 1)
plt.ylim(-2, 2)
plt.show()

for l in np.arange(0, 0.03, 0.004):
    for j in range(0, 16):
        mse_avg = []
        mae_avg = []
        err_l_avg = []
        for i in range(0, k):

            #Выбираем тестовые выборки
            X_test_k = X_train[(k-1)-i]
            y_test_k = y_train[(k-1)-i]

            #Исключаем из тренировочных выборок
            X_train_k = np.concatenate([np.delete(X_train, (k-1)-i)])
            y_train_k = np.concatenate([np.delete(y_train, (k-1)-i)])

            w = calculate_weights(X_train_k, y_train_k, j, l)
            predict = np.array(list(map(lambda x: polynom(x=x, w=w), X_test_k)))

            mse = mean_squared_error(y_test_k, predict)
            mse_avg.append(mse)

            mae = mean_absolute_error(y_test_k, predict)
            mae_avg.append(mae)

            err_lambda = err_l(y_test_k, predict, w, l)
            err_l_avg.append(err_lambda)

            plt.title('M = ' + str(j) + ';\n l = ' + str(l) + ';\n MSE = ' + str(mse) + ';\n MAE = ' + str(mae) + ';\n E_l = ' + str(err_lambda) + ';\n k = ' + str(i))

            plt.plot(z, func(z), 'b-')
            plt.plot(z, polynom(z, w), 'g-')
            plt.plot(X_train_k, y_train_k, 'ro')
            plt.plot(X_test_k, y_test_k, 'yo')
            plt.xlim(0, 1)
            plt.ylim(-2, 2)
            plt.show()

        results.append((j, l, np.average(mse_avg), np.average(mae_avg), np.average(err_l_avg)))

Лучшие параметры M и l

Для серднеквадратичной ошибки

In [ ]:
for t in (sorted(results, key=lambda t: t[2])):
    print('Degree = ', t[0], ' L = ', t[1], ' MSE = ', t[2])

Для абсолютной ошибки

In [ ]:
for t in (sorted(results, key=lambda t: t[3])):
    print('Degree = ', t[0], ' L = ', t[1], 'MAE = ', t[3])

Для ошибки кросс-валидации

In [ ]:
for t in (sorted(results, key=lambda t: t[4])):
    print('Degree = ', t[0], ' L = ', t[1], 'E_l = ', t[4])